![PyData_logo](./static/pydata-logo-madrid-2016.png)

# Remove Before Flight
## Analyzing Flight Safety Data with Python

###### Jesús Martos Carrizo
###### Alejandro Sáez Mollejo

### Introduction

#### Some figures 

![passengers](./static/passenger.jpg)
###### by Ryan McGuire

####  Flight Safety

#### Obejctive of this talk 

##### Disclaimer

#### Some remarks about

![pandas-logo](./static/pandas_logo.png)

#### Data source

<img src="./static/ntsb.jpg" style="width: 150px;"/>

__National Transportation Safety Board__ is charged with determining the probable cause of transportation accidents and promoting transportation safety, and assisting victims of transportation accidents and their families.

Data: http://www.ntsb.gov/_layouts/ntsb.aviation/index.aspx

### Loading & Cleaning data 

In [11]:
import pandas as pd
%matplotlib inline

In [2]:
# Some configuration options:
pd.set_option('max_columns', 70)
pd.set_option('max_rows', 50)

# to show all posiible options:
# pd.describe_option()

# to reset to default option value:
# pd.reset_option('max_columns')

# to show current value:
# pd.get_option('max_columns')

In [6]:
events = pd.read_csv("./data/events.csv",
                                    sep='|',
                                    index_col=0)  # Use eventid as index
Occurrences = pd.read_csv("./data/Occurrences1.csv",
                                    sep=';',
                                    index_col=0)  # Use eventid as index

C:\Users\jesus\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Aircraft_Key,Occurrence_No,Occurrence_Code,Phase_of_Flight,Altitude,lchg_date,lchg_userid
ev_id,,,,,,,
20001204X00000,1,1,350,560,0,12/12/2000,dbo
20001204X00000,1,2,180,552,0,12/12/2000,dbo
20001204X00000,1,3,320,574,0,12/12/2000,dbo
20001204X00001,1,1,130,572,0,12/12/2000,dbo
20001204X00001,1,2,171,513,0,12/12/2000,dbo
20001204X00002,1,1,310,513,0,27/04/2001,dbo
20001204X00003,1,1,240,531,0,27/04/2001,dbo
20001204X00003,1,2,250,564,0,27/04/2001,dbo
20001204X00003,1,3,230,553,0,27/04/2001,dbo


##### A first glipse of the data 

In [36]:
events.head(5)
# there is an extra row for ev_id with no data because we have used index_col=0 as an argument in the function read_csv()

,ntsb_no,ev_type,ev_date,ev_dow,ev_time,ev_tmzn,ev_city,ev_state,ev_country,ev_site_zipcode,ev_year,ev_month,mid_air,on_ground_collision,latitude,longitude,latlong_acq,apt_name,ev_nr_apt_id,ev_nr_apt_loc,apt_dist,apt_dir,apt_elev,wx_brief_comp,wx_src_iic,wx_obs_time,wx_obs_dir,wx_obs_fac_id,wx_obs_elev,wx_obs_dist,wx_obs_tmzn,light_cond,sky_cond_nonceil,sky_nonceil_ht,sky_ceil_ht,sky_cond_ceil,vis_rvr,vis_rvv,vis_sm,wx_temp,wx_dew_pt,wind_dir_deg,wind_dir_ind,wind_vel_kts,wind_vel_ind,gust_ind,gust_kts,altimeter,wx_dens_alt,wx_int_precip,metar,ev_highest_injury,inj_f_grnd,inj_m_grnd,inj_s_grnd,inj_tot_f,inj_tot_m,inj_tot_n,inj_tot_s,inj_tot_t,invest_agy,ntsb_docket,ntsb_notf_from,ntsb_notf_date,ntsb_notf_tm,fiche_number,lchg_date,lchg_userid,wx_cond_basic,faa_dist_office
ev_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20001208X07734,LAX97FA143,ACC,04/02/1997 12:00:00 AM,WE,2016,MST,GRAND CANYON,AZ,USA,86023,1997,4,N,N,,,NaN,,,OFAP,NaN,NaN,NaN,PARP,WFAC,1954,180,GCN,6606,21,MST,NDRK,SCAT,3600,6000,BKN,"0,00000000e+00",0,"1,00000000e+01",-2,-3,0,U,0,CALM,N,0,"2,90000000e+01",NaN,UNK,NaN,FATL,0,0,0,2,NaN,NaN,NaN,2,N,1890,NaN,04/02/1997 12:00:00 AM,1922,IMAGE,12/08/2000 12:13:19 PM,dbo,IMC,NaN
20021008X05297,CHI02LA299,ACC,09/29/2002 12:00:00 AM,Su,2100,CDT,Moorhead,MN,USA,56560,2002,9,N,N,465021N,0963947W,NaN,Moorhead Municipal,JKJ,ONAP,NaN,NaN,917,UNK,WFAC,2053,47,FAR,908,9,BST,NDRK,CLER,NaN,1300,OVC,NaN,NaN,"7,00000000e+00",14,12,160,Y,13,SPEC,Y,0,"2,97099991e+01",NaN,NaN,NaN,SERS,NaN,NaN,NaN,NaN,1,NaN,1,2,N,13246,FAA,10/03/2002 12:00:00 AM,945,DMS,04/23/2003 12:26:44 PM,NTSB\\MONR,VMC,Minneapolis FSDO
20040127X00111,LAX04LA103,ACC,10/12/2003 12:00:00 AM,Su,1640,MST,Scottsdale,AZ,USA,85255,2003,10,N,N,333722N,1115438W,NaN,Scottsdale,SDL,ONAP,NaN,NaN,1510,FULL,WFAC,1653,NaN,KSDL,1510,NaN,MST,DAYL,CLER,NaN,NaN,NONE,NaN,NaN,"1,00000000e+01",33,12,NaN,NaN,NaN,CALM,N,NaN,"2,98899994e+01",NaN,NaN,NaN,NONE,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,N,17586,FAA,01/22/2004 12:00:00 AM,1215,DMS,04/28/2005 02:59:26 PM,NTSB\\JOHB,VMC,WP-SDL-FSDO
20001213X34369,ANC86LA135,ACC,08/23/1986 12:00:00 AM,SA,1100,AKD,TALKEETNA,AK,USA,99676,1986,8,N,N,,,NaN,,,OFAP,"0,00000000e+00",0,0,UNK,PILO,0,0,,0,0,,DAYL,CLER,0,0,NONE,"0,00000000e+00",0,"5,00000000e+01",10,-18,0,U,0,UNK,U,0,NaN,NaN,UNK,NaN,NONE,0,0,0,NaN,NaN,3,NaN,NaN,N,891,NaN,08/23/1986 12:00:00 AM,1430,31238,01/02/2001 09:14:50 AM,dbo,VMC,NaN
20001213X35290,ANC87LA019,ACC,12/07/1986 12:00:00 AM,SU,1256,AST,SHAGELUK,AK,USA,99665,1986,12,N,N,,,NaN,,1,OFAP,"0,00000000e+00",0,0,UNK,PILO,0,0,,0,0,,DAYL,SCAT,1500,0,NONE,"0,00000000e+00",0,"4,00000000e+01",-1,-1,0,U,0,CALM,N,0,"2,90000000e+01",NaN,LGT,NaN,NONE,0,0,0,NaN,NaN,1,NaN,NaN,N,1400,NaN,12/07/1986 12:00:00 AM,1300,31791,01/02/2001 09:14:55 AM,dbo,VMC,NaN


In [48]:
events.columns
events = events.rename(columns = {'ev_dow':'Event day of the week','light_cond':'Light Conditions','wind_vel_kts':'Wind Speed(knots)',
                        'metar':'METAR Weather Report','vis_rvr':'Visibility Runway Visual Range (Feet)',
                        'vis_rvv':'Visibility Runway Visual Value (Statute Miles)','vis_sm':'Visibility (Statute Miles)',
                        'wx_cond_basic':'Basic Weather Conditions'})
events

,ntsb_no,ev_type,ev_date,Event day of the week,ev_time,ev_tmzn,ev_city,ev_state,ev_country,ev_site_zipcode,ev_year,ev_month,mid_air,on_ground_collision,latitude,longitude,latlong_acq,apt_name,ev_nr_apt_id,ev_nr_apt_loc,apt_dist,apt_dir,apt_elev,wx_brief_comp,wx_src_iic,wx_obs_time,wx_obs_dir,wx_obs_fac_id,wx_obs_elev,wx_obs_dist,wx_obs_tmzn,Light Conditions,sky_cond_nonceil,sky_nonceil_ht,sky_ceil_ht,sky_cond_ceil,Visibility Runway Visual Range (Feet),Visibility Runway Visual Value (Statute Miles),Visibility (Statute Miles),wx_temp,wx_dew_pt,wind_dir_deg,wind_dir_ind,Wind Speed(knots),wind_vel_ind,gust_ind,gust_kts,altimeter,wx_dens_alt,wx_int_precip,METAR Weather Report,ev_highest_injury,inj_f_grnd,inj_m_grnd,inj_s_grnd,inj_tot_f,inj_tot_m,inj_tot_n,inj_tot_s,inj_tot_t,invest_agy,ntsb_docket,ntsb_notf_from,ntsb_notf_date,ntsb_notf_tm,fiche_number,lchg_date,lchg_userid,Basic Weather Conditions,faa_dist_office
ev_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20001208X07734,LAX97FA143,ACC,04/02/1997 12:00:00 AM,WE,2016,MST,GRAND CANYON,AZ,USA,86023,1997,4,N,N,,,NaN,,,OFAP,NaN,NaN,NaN,PARP,WFAC,1954,180,GCN,6606,21,MST,NDRK,SCAT,3600,6000,BKN,"0,00000000e+00",0,"1,00000000e+01",-2,-3,0,U,0,CALM,N,0,"2,90000000e+01",NaN,UNK,NaN,FATL,0,0,0,2,NaN,NaN,NaN,2,N,1890,NaN,04/02/1997 12:00:00 AM,1922,IMAGE,12/08/2000 12:13:19 PM,dbo,IMC,NaN
20001214X38462,MIA86LA045,ACC,12/18/1985 12:00:00 AM,WE,1300,EST,LAKE CITY,FL,USA,32055,1985,12,N,N,,,NaN,,,OFAP,"0,00000000e+00",0,0,UNK,WFAC,1250,0,GNV,152,0,EST,DAYL,CLER,0,10000,BKN,"0,00000000e+00",0,"1,50000000e+01",16,2,60,U,5,UNK,N,0,"3,00000000e+01",100,UNK,NaN,MINR,0,0,0,NaN,1,NaN,NaN,1,N,2434,NaN,12/18/1985 12:00:00 AM,1415,29842,01/02/2001 09:15:17 AM,dbo,VMC,NaN
20001214X39736,NYC84LA176,ACC,05/16/1984 12:00:00 AM,WE,1515,EDT,SAYVILLE,NY,USA,,1984,5,N,N,,,NaN,L. I. MCARTHUR,,OFAP,"6,00000000e+00",240,NaN,UNK,PILO,0,0,,0,0,,DAYL,UNK,0,0,UNK,"0,00000000e+00",0,"0,00000000e+00",NaN,NaN,0,U,0,UNK,U,0,NaN,NaN,UNK,NaN,NONE,0,0,0,NaN,NaN,1,NaN,NaN,N,1528,NaN,NaN,0,25338,01/02/2001 09:15:26 AM,dbo,VMC,NaN
20001213X26058,NYC88LA152,ACC,06/01/1988 12:00:00 AM,WE,850,EDT,UTICA,NY,USA,13502,1988,6,N,N,,,NaN,,,OFAP,"0,00000000e+00",0,NaN,UNK,PILO,0,0,,0,0,,DAYL,UNK,0,3800,OVC,"0,00000000e+00",0,"3,00000000e+00",21,NaN,290,U,5,UNK,U,0,NaN,NaN,LGT,NaN,NONE,0,0,0,NaN,NaN,1,NaN,NaN,N,1955,NaN,06/01/1988 12:00:00 AM,1115,38223,12/13/2000 11:51:24 AM,dbo,VMC,NaN
20001211X15032,ANC92LA115,ACC,07/29/1992 12:00:00 AM,WE,900,AKD,YAKUTAT,AK,USA,99689,1992,7,N,N,,,NaN,,,OFAP,"0,00000000e+00",0,NaN,UNK,WFAC,858,265,YAK,33,30,ADT,DAYL,SCAT,500,2500,BKN,"0,00000000e+00",0,"5,00000000e+00",12,10,110,U,14,UNK,U,0,"3,00000000e+01",NaN,LGT,NaN,FATL,0,0,0,1,NaN,NaN,NaN,1,N,475,NaN,07/29/1992 12:00:00 AM,1630,47938,12/11/2000 03:03:52 PM,dbo,UNK,NaN
20001206X01235,CHI94LA154,ACC,05/11/1994 12:00:00 AM,WE,800,CDT,ANTHONY,KS,USA,67003,1994,5,N,N,,,NaN,ANTHONY AIRPORT,ANY,ONAP,NaN,NaN,1340,UNK,PILO,0,0,,0,0,,DAYL,UNK,0,1300,BKN,"0,00000000e+00",0,"1,50000000e+01",16,NaN,0,V,0,LVAR,N,0,"3,00000000e+01",NaN,UNK,NaN,MINR,0,0,0,NaN,1,2,NaN,1,N,648,NaN,05/11/1995 12:00:00 AM,1035,IMAGE,12/06/2000 11:11:47 AM,dbo,VMC,NaN
20001213X33993,NYC86LA145,ACC,06/04/1986 12:00:00 AM,WE,815,EDT,PERKASIE,PA,USA,18944,1986,6,N,N,,,NaN,PENNRIDGE,N70,ONAP,"0,00000000e+00",0,540,UNK,PILO,0,0,,0,0,,DAYL,CLER,0,0,UNK,"0,00000000e+00",0,"0,00000000e+00",NaN,NaN,0,U,0,UNK,U,0,NaN,NaN,UNK,NaN,NONE,0,0,0,NaN,NaN,1,NaN,NaN,N,1311,NaN,06/04/1986 12:00:00 AM,1000,31706,01/02/2001 09:14:47 AM,dbo,VMC,NaN
20001212X23300,ANC90LA084,ACC,06/06/1990 12:00:00 AM,WE,1830,AKD,WASILLA,AK,USA,99687,1990,6,N,N,,,NaN,WLF LAKE,,ONAP,"0,00000000e+00",0,520,UNK,PILO,0,0,,0,0,,DAYL,CLER,0,0,NONE,"0,00000000e+00",0,"7,00000000e+00",18,-18,50,U,5,UNK,U,15,"3,00000000e+01",0,UNK,NaN,NONE,0,0,0,NaN,NaN,1,NaN,NaN,N,204,NaN,06/08/1990 12:00:00 AM,800,40923,12/12/2000 05:02:17 PM,dbo,VMC,NaN
20001213X26310,MIA88LA220,ACC,07/13/1988 12:00:00 AM,WE,1350,E

In [38]:
# Not really useful for the majority of columns
events.describe()

,ev_time,ev_year,apt_dir,apt_elev,wx_obs_time,wx_obs_dir,wx_obs_elev,wx_obs_dist,sky_nonceil_ht,sky_ceil_ht,vis_rvv,wx_temp,wx_dew_pt,wind_dir_deg,wind_vel_kts,gust_kts,wx_dens_alt,inj_f_grnd,inj_m_grnd,inj_s_grnd,inj_tot_f,inj_tot_m,inj_tot_n,inj_tot_s,inj_tot_t,ntsb_docket,ntsb_notf_tm
count,76535.000000,76848.000000,37020.000000,48146.000000,69909.000000,63990.000000,67552.000000,65299.000000,56454.000000,55382.000000,48754.000000,67043.000000,55773.000000,67986.000000,69080.000000,53729.000000,20314.000000,49522.000000,49526.000000,49513.000000,15253.000000,14347.000000,46273.000000,10793.000000,35217.000000,72695.000000,56995.000000
mean,1373.671196,1996.920427,49.827769,1055.010344,999.464661,103.528911,866.237728,11.339745,3255.918819,2930.506861,0.146039,19.850603,6.877970,161.884167,7.257426,3.650245,1772.910357,0.010157,0.013064,0.011209,2.813545,1.866732,7.574676,1.543037,2.451969,8586.409781,1043.181577
std,434.386422,101.770110,100.047186,1655.995540,720.507141,122.243560,3746.138486,30.473635,6264.894555,6349.779335,22.881972,17.341558,15.660181,117.043219,5.951389,10.485619,2706.629533,0.353946,0.393807,0.518030,11.025730,5.172820,31.982420,2.722736,8.733082,10805.425263,718.420703
min,0.000000,1948.000000,0.000000,-210.000000,0.000000,0.000000,-115.000000,0.000000,0.000000,0.000000,0.000000,-62.000000,-38.000000,0.000000,0.000000,0.000000,-30536.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,1100.000000,1988.000000,0.000000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,0.000000,45.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,945.000000,1.000000
50%,1400.000000,1996.000000,0.000000,434.000000,1153.000000,40.000000,128.000000,1.000000,0.000000,0.000000,0.000000,21.000000,8.000000,170.000000,7.000000,0.000000,500.000000,0.000000,0.000000,0.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2005.000000,1200.000000
75%,1700.000000,2005.000000,20.000000,1071.000000,1553.000000,200.000000,845.000000,16.000000,4000.000000,2700.000000,0.000000,27.000000,16.000000,260.000000,10.000000,0.000000,2370.750000,0.000000,0.000000,0.000000,2.000000,2.000000,2.000000,2.000000,2.000000,16468.500000,1600.000000
max,2400.000000,26733.000000,979.000000,13250.000000,2400.000000,360.000000,751635.000000,1526.000000,250000.000000,250000.000000,4444.000000,1652.000000,1646.000000,999.000000,210.000000,999.000000,26858.000000,44.000000,62.000000,71.000000,349.000000,380.000000,699.000000,111.000000,380.000000,34935.000000,2400.000000


In [39]:
# Lets gather some info about the data
events.info(max_cols=0)

<class 'pandas.core.frame.DataFrame'>
Index: 76880 entries, 20001208X07734 to 20150121X60411
Columns: 70 entries, ntsb_no to faa_dist_office
dtypes: float64(27), object(43)
memory usage: 41.6+ MB


In [40]:
# What does 41.6+ MB mean???
events.info(max_cols=0, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 76880 entries, 20001208X07734 to 20150121X60411
Columns: 70 entries, ntsb_no to faa_dist_office
dtypes: float64(27), object(43)
memory usage: 207.8 MB


In [41]:
events.memory_usage(deep=True).sum() / 1048  # KiB  with deep=False same as events.values.nbyte; what is the difference when is True?

202655.713740458

In [45]:
# Maybe there is a better way to select...
events = events[events['ev_type'] == 'ACC']
# options:
# events = events.where(events['ev_dow']=='WE')       # the return a DataFrame of the same shape as the original.When the logical 
                                                    # condition in False the result is the whole row filled with NaNs.
                                                    # By default, where returns a modified copy of the data. There is an 
                                                    # optional parameter inplace so that the original data can be modified
                                                    # without creating a copy

# events = events.where(events['ev_dow']=='WE', 0, inplace=True)     # same that above but choosing the result instead of NaN and 
                                                                 # without creating a copy

# events = events.mask(events['ev_dow']=='WE')      # mask is the inverse boolean operation of where

# events = events.query('ev_type == ACC')     #  query() method that allows selection using an expression. In pandas 0.18 is 
                                              # an experimental method. Can this become useful?

In [12]:
events.info(max_cols=0, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 73893 entries, 20001208X07734 to 20150121X60411
Columns: 70 entries, ntsb_no to faa_dist_office
dtypes: float64(27), object(43)
memory usage: 201.5 MB


In [13]:
events['ev_date'].head()

ev_id
20001208X07734    04/02/1997 12:00:00 AM
20021008X05297    09/29/2002 12:00:00 AM
20040127X00111    10/12/2003 12:00:00 AM
20001213X34369    08/23/1986 12:00:00 AM
20001213X35290    12/07/1986 12:00:00 AM
Name: ev_date, dtype: object

In [ ]:
%timeit pd.to_datetime(events['ev_date'], infer_datetime_format=True)

In [ ]:
%timeit pd.to_datetime(events['ev_date'], infer_datetime_format=False)

In [14]:
# Information about date format: 
# https://docs.python.org/3.5/library/datetime.html#strftime-and-strptime-behavior
date_format = "%m/%d/%Y %I:%M:%S %p"
% timeit pd.to_datetime(events['ev_date'], format=date_format)

1 loop, best of 3: 780 ms per loop


In [15]:
date_format = "%m/%d/%Y %I:%M:%S %p"
events['ev_date'] = pd.to_datetime(events['ev_date'], format=date_format)

In [16]:
events['ev_date'][0].dayofweek

2

In [18]:
# Convert column dtype to string, extract the time and pad with zeros
ev_time = events['ev_time'].astype(str).str.split('.').str[0].str.zfill(4)

pd.to_datetime(ev_time,
                         format="%H%M",  # format
                         errors='coerce')  # for nan values

ev_id
20001208X07734   1900-01-01 20:16:00
20021008X05297   1900-01-01 21:00:00
20040127X00111   1900-01-01 16:40:00
20001213X34369   1900-01-01 11:00:00
20001213X35290   1900-01-01 12:56:00
20001214X43255   1900-01-01 13:00:00
20010110X00143   1900-01-01 16:15:00
20001213X33703   1900-01-01 18:15:00
20001214X40436   1900-01-01 11:20:00
20001211X14218   1900-01-01 11:45:00
20001212X23966   1900-01-01 19:30:00
20001214X42435   1900-01-01 11:00:00
20030924X01588   1900-01-01 10:30:00
20120214X84853   1900-01-01 12:20:00
20001214X37578   1900-01-01 09:30:00
20001214X38462   1900-01-01 13:00:00
20020917X03633   1900-01-01 20:30:00
20081003X65757   1900-01-01 18:30:00
20110716X12058   1900-01-01 15:45:00
20001212X21991   1900-01-01 16:22:00
20001213X28279   1900-01-01 16:20:00
20060803X01083   1900-01-01 14:00:00
20080718X01072   1900-01-01 17:00:00
20001211X13648   1900-01-01 13:20:00
20001212X23150   1900-01-01 08:20:00
                         ...        
20001213X26120   1900-01-01 23:0

In [19]:
events['ev_time'] = pd.to_datetime(ev_time, format="%H%M", errors='coerce') 

In [20]:
events['ev_date'][0:5].dt.date.astype(str) + ' ' + events['ev_time'][0:5].dt.time.astype(str)

ev_id
20001208X07734    1997-04-02 20:16:00
20021008X05297    2002-09-29 21:00:00
20040127X00111    2003-10-12 16:40:00
20001213X34369    1986-08-23 11:00:00
20001213X35290    1986-12-07 12:56:00
dtype: object

In [21]:
# FIXME
time_series = pd.to_datetime(events['ev_date'].dt.date.astype(str) + ' ' + events['ev_time'].dt.time.astype(str), 
                         format="%Y-%m-%d %H:%M:%S",
                         errors='coerce')  # !! WATCH OUT I am loosing some dates due to missing times...

In [22]:
pd.to_datetime('2002-09-29' + ' ' + 'NaT', 
                               format="%Y-%m-%d %H:%M:%S",
                               errors='coerce')

NaT

In [23]:
pd.to_datetime('2002-09-29' + ' ' + '00', 
                               format="%Y-%m-%d %H:%M:%S",
                               errors='coerce')

Timestamp('2002-09-29 00:00:00')

In [25]:
# isnull detects 'NaT'
pd.isnull(events['ev_time']).sum()

194

In [26]:
# Aha! so the same...
pd.isnull(time_series).sum()

194

### Data Analysis 

#### Accidents location 

#### Accident trends 

In [13]:
a = Occurrences['Phase_of_Flight'].value_counts()
b = a[0:7]
b = b.rename_axis({572:'LANDING-ROLL',571:'LANDING - FLARE/TOUCHDOWN',553:'DESCENT - UNCONTROLLED',
                  552:'DESCENT - EMERGENCY (plt initiated)',522:'TAKEOFF - INITIAL CLIMB',580:'MANEUVERING (includes buzzing)',
                  540:'CRUISE (includes low altitude straight and level flight)'})

b

LANDING-ROLL                                                21339
LANDING - FLARE/TOUCHDOWN                                   14871
DESCENT - UNCONTROLLED                                      13874
DESCENT - EMERGENCY (plt initiated)                         10714
TAKEOFF - INITIAL CLIMB                                      9746
MANEUVERING (includes buzzing)                               6607
CRUISE (includes low altitude straight and level flight)     6300
Name: Phase_of_Flight, dtype: int64

#### When do accidents occur? 

#### Other Questions... 

### Conclusions 

# Thanks for yor attention! 

![PyData_logo](./static/pydata-logo-madrid-2016.png)

## Any Questions?


---


In [1]:
# Notebook style
from IPython.core.display import HTML
css_file = './static/style.css'
HTML(open(css_file, "r").read())